First of all, import all needed packages, particularly: **torch** and **torchvision**.

In [16]:
# Original Source: https://github.com/pytorch/examples/blob/master/mnist/main.py
# Code from: https://github.com/jiapei100/PyTorchZeroToAll/blob/master/10_1_cnn_mnist.py

from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

Loading MNIST dataset. You'll see 4 datasets are downloaded during the running process.
* Downloading http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
* Downloading http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
* Downloading http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
* Downloading http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz


In [17]:

# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


Afterwards, define CNN


In [18]:

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x, dim=1)


model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


Define training function

In [19]:

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))


Define testing function

In [20]:

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += F.nll_loss(output, target, size_average=False).data[0]
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


Train and test

In [ ]:

for epoch in range(1, 10):
    train(epoch)
    test()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.313586
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.281689
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.297252
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.280714
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.270877
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.235533
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.230326
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.191298
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.165507
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.094295
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.977994
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.795815
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.640058
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.353927
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.186284
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.911913
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.701897
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.708441
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.927225
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.762540
Train Epoch: 1 [12800/60000 (